<a href="https://colab.research.google.com/github/SMunira38/Final-Project/blob/main/RPDC%20120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:

import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.models import vgg19
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
from torch.utils.data import random_split

In [17]:
# Set random seed for reproducibility
torch.manual_seed(42)


In [18]:

# Define paths for the dataset
data_dir = '/content/drive/MyDrive/rice_leaf_diseases'
batch_size = 32

In [19]:

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
# Load the entire dataset from Google Drive
dataset = torchvision.datasets.ImageFolder(root=data_dir, transform=transform)

In [20]:
import random
from tabulate import tabulate

# Create dictionaries to store data for train, validation, and test splits for each class
train_class_data = {class_idx: [] for class_idx in range(len(dataset.classes))}
val_class_data = {class_idx: [] for class_idx in range(len(dataset.classes))}
test_class_data = {class_idx: [] for class_idx in range(len(dataset.classes))}

# Define the proportions for train, validation, and test
train_proportion = 0.70
val_proportion = 0.15
test_proportion = 0.15

# Split the data for each class while ensuring balance
for class_idx in range(len(dataset.classes)):
    class_data = [(data, target) for data, target in dataset if target == class_idx]
    random.shuffle(class_data)  # Shuffle the data for each class

    total_samples = len(class_data)
    train_size = int(train_proportion * total_samples)
    val_test_size = int(val_proportion * total_samples)

    train_class_data[class_idx] = class_data[:train_size]
    val_class_data[class_idx] = class_data[train_size:(train_size + val_test_size)]
    test_class_data[class_idx] = class_data[(train_size + val_test_size):(train_size + 2*val_test_size)]

# Merge data for all classes into train, validation, and test sets
train_data = [item for sublist in train_class_data.values() for item in sublist]
val_data = [item for sublist in val_class_data.values() for item in sublist]
test_data = [item for sublist in test_class_data.values() for item in sublist]

# Define data loaders for each set
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)


# Print the counts for each class in each set in a tabular form
print("Class-wise Data:")
table_data = []
for class_idx in range(len(dataset.classes)):
    class_name = dataset.classes[class_idx]
    train_count = len(train_class_data[class_idx])
    val_count = len(val_class_data[class_idx])
    test_count = len(test_class_data[class_idx])
    table_data.append([class_name, train_count, val_count, test_count])

# Create headers for the table
headers = ["Class", "Train Sample", "Validation Sample", "Test Sample"]
# Print the tabulated data
print(tabulate(table_data, headers, tablefmt='grid'))

Class-wise Data:
+-----------------------+----------------+---------------------+---------------+
| Class                 |   Train Sample |   Validation Sample |   Test Sample |
+=======================+================+=====================+===============+
| Bacterial leaf blight |             28 |                   6 |             6 |
+-----------------------+----------------+---------------------+---------------+
| Brown spot            |             28 |                   6 |             6 |
+-----------------------+----------------+---------------------+---------------+
| Leaf smut             |             28 |                   6 |             6 |
+-----------------------+----------------+---------------------+---------------+


In [21]:
   # Get the total number of classes and total number of images
total_classes = len(dataset.classes)
total_images = len(dataset)

# Get the total number of images in the train, validation, and test sets
total_train_images = len(train_data)
total_val_images = len(val_data)
total_test_images = len(test_data)

print(f"Total Classes: {total_classes}")
print(f"Total Images: {total_images}")

print(f"Total Train Images: {total_train_images}")
print(f"Total Validation Images: {total_val_images}")
print(f"Total Test Images: {total_test_images}")

Total Classes: 3
Total Images: 120
Total Train Images: 84
Total Validation Images: 18
Total Test Images: 18


In [22]:
num_classes=len(dataset.classes)
# Load the pre-trained VGG19 model and modify it for your classification task

model = vgg19(pretrained=True)
model.classifier[6] = nn.Linear(4096, num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [23]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [24]:
# Training loop
num_epochs = 15
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
test_accuracies = []  # List to store testing accuracies for each epoch

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions_train = 0
    total_samples_train = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_samples_train += labels.size(0)
        correct_predictions_train += (predicted == labels).sum().item()

    train_losses.append(running_loss / len(train_loader))
    train_accuracy = correct_predictions_train / total_samples_train
    train_accuracies.append(train_accuracy)

    # Validation loop
    model.eval()
    running_val_loss = 0.0
    correct_predictions_val = 0
    total_samples_val = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_samples_val += labels.size(0)
            correct_predictions_val += (predicted == labels).sum().item()

    val_losses.append(running_val_loss / len(val_loader))
    val_accuracy = correct_predictions_val / total_samples_val
    val_accuracies.append(val_accuracy)

    # Testing loop
    model.eval()
    correct_predictions_test = 0
    total_samples_test = 0
    actual_labels_test = []
    predicted_labels_test = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_samples_test += labels.size(0)
            correct_predictions_test += (predicted == labels).sum().item()

            # Append actual and predicted labels for testing
            actual_labels_test += labels.cpu().numpy().tolist()
            predicted_labels_test += predicted.cpu().numpy().tolist()

    test_accuracy = correct_predictions_test / total_samples_test
    test_accuracies.append(test_accuracy)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {train_losses[-1]:.6f} - Train Acc: {train_accuracy * 100:.2f}% - Val Loss: {val_losses[-1]:.6f} - Val Acc: {val_accuracy * 100:.2f}% - Test Acc: {test_accuracy * 100:.2f}%")


print("Finished Training")

Epoch [1/15] - Train Loss: 1.176919 - Train Acc: 29.76% - Val Loss: 1.134700 - Val Acc: 33.33% - Test Acc: 33.33%
Epoch [2/15] - Train Loss: 1.032605 - Train Acc: 44.05% - Val Loss: 0.931111 - Val Acc: 55.56% - Test Acc: 55.56%
Epoch [3/15] - Train Loss: 0.834751 - Train Acc: 64.29% - Val Loss: 0.731047 - Val Acc: 72.22% - Test Acc: 77.78%
Epoch [4/15] - Train Loss: 0.604448 - Train Acc: 84.52% - Val Loss: 0.588530 - Val Acc: 77.78% - Test Acc: 88.89%
Epoch [5/15] - Train Loss: 0.375098 - Train Acc: 91.67% - Val Loss: 0.545603 - Val Acc: 83.33% - Test Acc: 83.33%
Epoch [6/15] - Train Loss: 0.230270 - Train Acc: 94.05% - Val Loss: 0.535916 - Val Acc: 88.89% - Test Acc: 88.89%
Epoch [7/15] - Train Loss: 0.149812 - Train Acc: 97.62% - Val Loss: 0.629787 - Val Acc: 83.33% - Test Acc: 88.89%
Epoch [8/15] - Train Loss: 0.157191 - Train Acc: 95.24% - Val Loss: 0.519804 - Val Acc: 83.33% - Test Acc: 88.89%
Epoch [9/15] - Train Loss: 0.054282 - Train Acc: 98.81% - Val Loss: 0.600683 - Val Acc: 